# Part I: 文法四元组提取 Grammatical quadruple extraction

In [1]:
grammar_string = "E ::= E + T | T\nT ::= T * F | F\nF ::= ( E ) | i"

### 定义文法提取函数

In [5]:
def add_production(non_terminal, production, productions):
    if non_terminal in productions:
        productions[non_terminal].append(production)
    else:
        productions[non_terminal] = [production]

def extract_from_line(s):
    left,right = s.split('::=')
    return left,right

def extract_rules_from_right(ls,rs,p):
    mid_re = rs.split('|')
    for item in mid_re:
        item_ts = []
        for item_t in item:
            item_ts += [item_t]
        add_production(ls,item_ts,p)

def extract_grammar_components(gs):
    production = {}
    start = ''
    terminators = []
    non_terminators = []
    gs = gs.replace(' ','')
    mid_s = gs.split("\n")
    for item_s in mid_s:
        left,right = extract_from_line(item_s)
        extract_rules_from_right(left, right, production)
        if ('Z' >= left >= 'A') and left not in terminators:
            terminators.append(left)
        if start == '':
            start = left
        for item_r in right:
            if item_r == '|': continue
            if (item_r > 'Z' or item_r < 'A') and item_r not in non_terminators:
                non_terminators.append(item_r)
    return terminators,non_terminators,production,start

**检验输出**

In [6]:
Vt,Vn,P,S = extract_grammar_components(grammar_string)
print(grammar_string,'\n')
print(Vt)
print(Vn)
print(P)
print(S)

E ::= E + T | T
T ::= T * F | F
F ::= ( E ) | i 

['E', 'T', 'F']
['+', '*', '(', ')', 'i']
{'E': [['E', '+', 'T'], ['T']], 'T': [['T', '*', 'F'], ['F']], 'F': [['(', 'E', ')'], ['i']]}
E


# Part II 文法压缩 Grammatical compression

### 测试数据

In [7]:
extra_string = "Z ::= Be\nA ::= Ae\nA ::= e\nB ::= Ce\nB ::= Af\nC ::= Cf\nD ::= f\nE ::= E"

In [8]:
e_Vt,e_Vn,e_P,e_S = extract_grammar_components(extra_string)
print(extra_string,'\n')
print(e_Vt)
print(e_Vn)
print(e_P)
print(e_S)

Z ::= Be
A ::= Ae
A ::= e
B ::= Ce
B ::= Af
C ::= Cf
D ::= f
E ::= E 

['Z', 'A', 'B', 'C', 'D', 'E']
['e', 'f']
{'Z': [['B', 'e']], 'A': [['A', 'e'], ['e']], 'B': [['C', 'e'], ['A', 'f']], 'C': [['C', 'f']], 'D': [['f']], 'E': [['E']]}
Z


### 删除U::=U 型的规则

In [9]:
def print_productions(productions):
    for key in productions:
        tss = ""
        for item in productions[key]:
            for item_s in item:
                for item_t in item_s:
                    tss += item_t
            tss += '|'
        print(f"{key}::=",tss[:-1])

In [10]:
def clean_UtoU(production):
    # 创建一个新的字典来保存修改后的产生式
    cleaned_production = {}
    for key, value in production.items():
        for item in value:
            if key in item and len(item) == 1:
                continue
            add_production(key, item, cleaned_production)
    return cleaned_production
# da = {'Z': [['B', 'e']], 'A': [['A', 'e'], ['e']], 'B': [['C', 'e'], ['A', 'f'],['B']], 'C': [['C', 'f']], 'D': [['D']]}
print_productions(clean_UtoU(e_P))

Z::= Be
A::= Ae|e
B::= Ce|Af
C::= Cf
D::= f


### 删除U::=u 型的规则
- 条件1：从S开始，对所有左部为有标记非终结符的生成式的右部的非终结符进行标记，反复进行这一过程至再无满足条件的生成式
- 条件2：对所有右部只有终结符的生成式的左部的非终结符进行标记。对所有左部为非终结符且右部只有终结符和标记非终结符的生成式的左部的非终结符进行标记，反复执行这一过程至再无满足条件的生成式
- 反复执行条件1和条件2的方法直至再无满足删除条件的生成式

In [11]:
import copy


def clean_Utou1(production, start):
    end_production = copy.deepcopy(production)  # 复制原始产生式字典
    marks = [start]
    start_list = production[start]
    # 遍历起始符号的产生式，标记所有出现在其中的非终结符
    for item in start_list:
        for item_s in item:
            if 'A' <= item_s <= 'Z' and item_s not in marks:
                marks.append(item_s)
    tag1 = True
    tag2 = True
    while tag1 or tag2:
        tag1 = False
        tag2 = False
        # 根据标记集合更新标记
        for key in production:
            if key in marks:
                continue
            k_list = production[key]
            for item in k_list:
                for item_s in item:
                    if 'A' <= item_s <= 'Z' and item_s in marks:
                        tag1 = True
                        marks.append(key)
        for key in marks:
            k_list = production[key]
            for item in k_list:
                for item_s in item:
                    if 'A' <= item_s <= 'Z' and item_s not in marks:
                        tag2 = True
                        marks.append(item_s)
    # 根据标记集合删除未标记的产生式
    for key in end_production.copy():
        if key not in marks:
            del end_production[key]
    return end_production

def clean_Utou2(production):
    end_production = {}
    marks = []
    for key in production:
        for item in production[key]:
            tag = True
            # 检测是否有非终结符
            for item_s in item:
                if 'A' <= item_s <= 'Z':
                    tag = False
                    break
            # 没有非终结符才能标记
            if tag: 
                marks.append(key)
                add_production(key,item,end_production)
    tag1 = True
    while tag1:
        tag1 = False
        for key in production:
            for item in production[key]:
                num_no_mark = False
                # 检测右部是否只有被标记的非终结符和终结符
                for item_s in item:
                    if 'A' <= item_s <= 'Z' and item_s not in marks:
                        num_no_mark = True
                        break
                # 右部只有被标记的非终结符和终结符时才能标记左部,当然前提是这条没被记录过
                if not num_no_mark and (key not in end_production or item not in end_production[key]):
                    tag1 = True
                    marks.append(key)
                    add_production(key,item,end_production)
    return end_production       

**检验输出**

In [12]:
print_productions(e_P)
print("")
clu = clean_Utou1(e_P,e_S)
print_productions(clu)

Z::= Be
A::= Ae|e
B::= Ce|Af
C::= Cf
D::= f
E::= E

Z::= Be
A::= Ae|e
B::= Ce|Af
C::= Cf


In [13]:
print_productions(clean_Utou2(clu))

A::= e|Ae
B::= Af
Z::= Be


### 自动化

In [14]:
def auto_clean(production,start):
    end_production = clean_UtoU(production)
    tag = True
    while tag:
        end_production = clean_Utou1(end_production,start)
        if end_production == end_production:
            tag = False
        end_production = clean_Utou2(end_production)
        if end_production == end_production:
            tag = False
    return end_production

检验输出

In [15]:
print_productions(e_P)
print('')
acl = auto_clean(e_P,e_S)
print_productions(acl)

Z::= Be
A::= Ae|e
B::= Ce|Af
C::= Cf
D::= f
E::= E

A::= e|Ae
B::= Af
Z::= Be


# Part III 消除左递归 Eliminate left recursion

### 测试数据

In [53]:
left_recursion_string = "S ::= Sa | Tbc | Td\nT ::= Se | gh"
left_recursion_string_1 = "A ::= Ba | Cb | c\nB ::= da | Ae | f\nC ::= Bg | h"
print(left_recursion_string,'\n')
print(left_recursion_string_1)

S ::= Sa | Tbc | Td
T ::= Se | gh 

A ::= Ba | Cb | c
B ::= da | Ae | f
C ::= Bg | h


In [54]:
l_Vt_1,l_Vn_1,l_P_1,l_S_1 = extract_grammar_components(left_recursion_string_1)
print(l_Vt_1)
print(l_Vn_1)
print(l_P_1)
print(l_S_1)

['A', 'B', 'C']
['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']
{'A': [['B', 'a'], ['C', 'b'], ['c']], 'B': [['d', 'a'], ['A', 'e'], ['f']], 'C': [['B', 'g'], ['h']]}
A


In [55]:
l_Vt,l_Vn,l_P,l_S = extract_grammar_components(left_recursion_string)
print(l_Vt)
print(l_Vn)
print(l_P)
print(l_S)

['S', 'T']
['a', 'b', 'c', 'd', 'e', 'g', 'h']
{'S': [['S', 'a'], ['T', 'b', 'c'], ['T', 'd']], 'T': [['S', 'e'], ['g', 'h']]}
S


### 函数方法实现

In [56]:
def remove_production(non_terminal, production, productions):
    if non_terminal in productions and production in productions[non_terminal]:
        productions[non_terminal].remove(production)
        if not productions[non_terminal]:  # If the set becomes empty, remove the key
            del productions[non_terminal]

def eliminate_rules(key_item,value_item,production):
    end_production = copy.deepcopy(production)
    mid_key_item = key_item
    end_key_item = mid_key_item + "'"
    mid_value_item = value_item.copy()
    end_value_item = value_item.copy()
    # 先改后删
    for i in range(len(end_value_item)):
        if end_value_item[i] == mid_key_item:
            end_value_item.pop(i)
            end_value_item.append(end_key_item)
            break
    for item in end_production[mid_key_item]:
        if item == mid_value_item: continue
        if mid_key_item in item: continue
        if end_key_item in item: continue
        item.append(end_key_item)
    remove_production(mid_key_item,mid_value_item,end_production)
    add_production(end_key_item,end_value_item,end_production)
    if ['ε'] not in end_production[end_key_item]:
        add_production(end_key_item,['ε'],end_production)
    else:
        remove_production(end_key_item,['ε'],end_production)
        add_production(end_key_item,['ε'],end_production)
    return end_production
        
def auto_eliminate_rules(productions):
    end_productions = copy.deepcopy(productions)
    tag = True
    while tag:
        tag = False
        for key in end_productions:
            for item in end_productions[key]:
                # 消去规则左递归
                if key in item and len(key) == 1:
                    tag = True
                    end_productions = eliminate_rules(key,item,end_productions)  
    return end_productions


def eliminate_grammar(productions):
    mid_productions = copy.deepcopy(productions)
    end_productions = copy.deepcopy(productions)
    mid_changed_keys_list = []
    for key in productions:
        if "'" in key:
            mid_changed_keys_list.append(key.replace("'",''))
    flg = False
    for key in mid_productions:
        if key in mid_changed_keys_list: continue
        for item in mid_productions[key]:
            tag = False
            for i in range(len(item)):
                if item[i] in mid_changed_keys_list:
                    tag = True
                    flg = True
                    for item_i in mid_productions[item[i]]:
                        end_item = item.copy()
                        end_item.pop(i)
                        end_item[i:i] = item_i
                        add_production(key,end_item,end_productions)
            if tag:
                remove_production(key,item,end_productions)
    return end_productions,flg           

**检验输出**

In [57]:
print_productions(l_P)
print('\n')
l_Pr = auto_eliminate_rules(l_P)
print_productions(l_Pr)
print('\n')
l_Pg,flg = eliminate_grammar(l_Pr)
print_productions(l_Pg)
print(flg,'\n')
l_Pg2,flg = eliminate_grammar(l_Pg)
print_productions(l_Pg2)
print(flg,'\n')

S::= Sa|Tbc|Td
T::= Se|gh


S::= TbcS'|TdS'
T::= Se|gh
S'::= aS'|ε


S::= TbcS'|TdS'
T::= gh|TbcS'e|TdS'e
S'::= aS'|ε
True 

S::= TbcS'|TdS'
T::= gh|TbcS'e|TdS'e
S'::= aS'|ε
False 



### 自动化

In [58]:
def auto_eliminate_left_recursion(productions):
    end_productions = copy.deepcopy(productions)
    while True:  
        end_productions = auto_eliminate_rules(end_productions)
        end_productions,flg = eliminate_grammar(end_productions)
        if not flg:
            break
    return end_productions

**检验输出**

In [59]:
print_productions(l_P)
l_Pc = auto_eliminate_left_recursion(l_P)
print('')
print_productions(l_Pc)

S::= Sa|Tbc|Td
T::= Se|gh

S::= TbcS'|TdS'
T::= ghT'
S'::= aS'|ε
T'::= bcS'eT'|dS'eT'|ε


In [60]:
print_productions(P)
Pc = auto_eliminate_left_recursion(P)
print('')
print_productions(Pc)

E::= E+T|T
T::= T*F|F
F::= (E)|i

E::= TE'
T::= FT'
F::= iF'
E'::= ε|+iF'T'E'
T'::= ε|*iF'T'
F'::= (T'E')F'|ε


# Part IV 最左推导 Leftmost derivation

### 测试数据

In [61]:
print_productions(P)

E::= E+T|T
T::= T*F|F
F::= (E)|i


In [62]:
derivation_string = "i + i * i"
derivation_string_1 = "( i + i ) * i"
print(derivation_string)
print(derivation_string_1)

i + i * i
( i + i ) * i


### 构建推导方法函数

In [63]:
def input_judgment(derived_string, key, end_string,pos):
    while True:
        answer = input("\nDerive {} with {} -> {}? pos: {} (y/n): \n".format(derived_string, key, end_string,pos))
        if answer == 'y':
            return False
        elif answer == 'n':
            return True
        else:
            print("Please enter 'y' or 'n'")
            continue
    # 如果回答不是'y'，则跳过当前产生式


def find_value_in_list(lst, value):
    for i, item in enumerate(lst):
        if item == value:
            return i
    return -1


def auto_derivation(input_str, productions, start_symbol):
    derived_string = [start_symbol]  # 初始时将起始符号作为推导的起点
    result = ""  # 存储推导结果
    tree = {}  # 存储语法分析树
    # 逐步推导直到无法再推导为止
    while derived_string:
        derived = False  # 标志是否成功推导
        # 遍历所有产生式
        for key in productions:
            # 如果当前推导串中包含产生式左部，则问询是否进行替换
            if key in derived_string:
                for item in productions[key]:
                    # 问询是否替换
                    end_string = "".join(item)
                    mid_string = "".join(derived_string)

                    pos = find_value_in_list(derived_string, key)
                    if input_judgment(mid_string, key, end_string,pos + 1):
                        continue  # 如果回答不是'y'，则跳过当前产生式
                    
                    # 将当前推导过程添加到结果中
                    result += " => {} ({} -> {})\n".format(mid_string, key, end_string)

                    # 进行替换
                    derived_string[pos:pos+1] = item
                    derived = True  # 表示成功推导
                    
                    end_item = [pos + 1,item]
                    # 更新语法分析树
                    if key in tree:
                        tree[key].append(end_item)
                    else:
                        tree[key] = [end_item]
                    break  # 结束当前循环，重新开始检查推导串

                # 如果成功推导，就跳出外层循环
                if derived:
                    break

        # 如果无法再进行推导，则退出循环
        if derived_string == input_str:
            mid_string_out = "".join(derived_string)
            result += " => {}\n".format(mid_string_out)
            break
        if not derived:
            mid_string_out = "".join(derived_string)
            result += " => {}\n".format(mid_string_out)
            break

    return result, tree


检验输出

In [64]:
rel, tre = auto_derivation(derivation_string, P, S)
print(rel)
print(tre)

Please enter 'y' or 'n'
 => E (E -> E+T)
 => E+T (E -> E+T)
 => E+T+T (E -> T)
 => T+T+T (T -> T*F)
 => T*F+T+T (T -> F)
 => F*F+T+T (T -> F)
 => F*F+F+T (T -> F)
 => F*F+F+F (F -> i)
 => i*F+F+F (F -> i)
 => i*i+F+F (F -> i)
 => i*i+i+F (F -> i)
 => i*i+i+i

{'E': [[1, ['E', '+', 'T']], [1, ['E', '+', 'T']], [1, ['T']]], 'T': [[1, ['T', '*', 'F']], [1, ['F']], [5, ['F']], [7, ['F']]], 'F': [[1, ['i']], [3, ['i']], [5, ['i']], [7, ['i']]]}


# Part V 语法分析树构建 Parse Tree

### 函数方法实现

In [65]:
def generate_tree(tree, start):
    mid_tree = copy.deepcopy(tree)
    pre_string = "_" * 10 + start
    result = pre_string + '\n'
    tag = True
    while tag:
        tag = False
        cur_string = ""
        j = 0
        for i in range(len(pre_string)):
            if pre_string[i] == '_': continue
            j += 1
            if pre_string[i] in mid_tree:
                tag = True
                item_left, item_rigth = mid_tree[pre_string[i]][0]
                if j == item_left:
                    mid_tree[pre_string[i]].pop(0)
                    cur_string = pre_string[:i] + "__________".join(item_rigth) + pre_string[i + 1:]
                    break
        result += cur_string + '\n'
        pre_string = cur_string
    return result


**检验输出**

In [66]:
# 测试
print("语法分析树：")
ans = generate_tree(tre, S)
print(ans)

语法分析树：
__________E
__________E__________+__________T
__________E__________+__________T__________+__________T
__________T__________+__________T__________+__________T
__________T__________*__________F__________+__________T__________+__________T
__________F__________*__________F__________+__________T__________+__________T
__________i__________*__________F__________+__________T__________+__________T
__________i__________*__________i__________+__________T__________+__________T
__________i__________*__________i__________+__________F__________+__________T
__________i__________*__________i__________+__________i__________+__________T
__________i__________*__________i__________+__________i__________+__________F
__________i__________*__________i__________+__________i__________+__________i


